# Day 9 - Finding the sum, again, with a running series

- https://adventofcode.com/2020/day/9

This looks to be a variant of the [day 1, part 1 puzzle](./Day%2001.ipynb); finding the sum of two numbers in a set. Only now, we have to make sure we know what number to remove as we progres! This calls for a _sliding window_ iterator really, where we view the whole series through a slit X entries wide as it moves along the inputs.

As this puzzle is easier with a set of numbers, I create a sliding window of size `preamble + 2`, so we have access to the value to be removed and the value to be checked, at the same time; to achieve this, I created a window function that takes an _offset_, where you can take `offset` fewer items at the start, then have the window grow until it reaches the desired size:


In [1]:
from collections import deque
from itertools import islice
from typing import Iterable, Iterator, TypeVar

T = TypeVar("T")


def window(iterable: Iterable[T], n: int = 2, offset: int = 0) -> Iterator[deque[T]]:
    it = iter(iterable)
    queue = deque(islice(it, n - offset), maxlen=n)
    yield queue
    append = queue.append
    for elem in it:
        append(elem)
        yield queue


def next_invalid(numbers: Iterable[int], preamble: int = 25) -> int:
    it = window(numbers, preamble + 2, 2)
    pool = set(next(it))
    for win in it:
        to_check = win[-1]
        if len(win) == preamble + 2:
            # remove the value now outside of our preamble window
            pool.remove(win[0])

        # validate the value can be created from a sum
        for a in pool:
            b = to_check - a
            if b == a:
                continue
            if b in pool:
                # number validated
                break
        else:
            # no valid sum found
            return to_check

        pool.add(to_check)


test = [
    int(v)
    for v in """\
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
""".split()
]
assert next_invalid(test, 5) == 127

In [2]:
import aocd

number_stream = [int(v) for v in aocd.get_data(day=9, year=2020).split()]

In [3]:
print("Part 1:", next_invalid(number_stream))

Part 1: 18272118


## Part 2

To solve the second part, you need a _dynamic_ window size over the input stream, and a running total. When the running total equals the value from part 1, we can then take the min and max values from the window.

- While the running total is too low, grow the window one stap and add the extra value to the total
- If the running total is too high, remove a value at the back of the window from the running total, and shrink that side of the window by one step.

With the Python `deque` (double-ended queue) already used in part one, this is a trivial task to achieve:


In [4]:
def find_weakness(numbers: Iterable[int], preamble: int = 25) -> int:
    invalid = next_invalid(numbers, preamble)
    it = iter(numbers)
    total = next(it)
    window = deque([total])
    while total != invalid and window:
        if total < invalid:
            window.append(next(it))
            total += window[-1]
        else:
            total -= window.popleft()
    if not window:
        raise ValueError("Could not find a weakness")
    return min(window) + max(window)


assert find_weakness(test, 5) == 62

In [5]:
print("Part 2:", find_weakness(number_stream))

Part 2: 2186361
